In [292]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

import ipykernel.iostream
import sys

import urllib3.contrib.pyopenssl

In [273]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

## Attribution: https://stackoverflow.com/questions/43846689/sslerror-bad-handshake-syscallerror54-econnreset

In [24]:
links = pd.read_csv('../data/links.csv', names = ['link'], skiprows=1)

In [25]:
links

,link
0,https://onebite.app/restaurant/pizza-hut-adana...
1,https://onebite.app/restaurant/del-sole-01-472...
2,https://onebite.app/restaurant/lava-lounge-piz...
3,https://onebite.app/restaurant/pizzaria-cosa-n...
4,https://onebite.app/restaurant/olivia-tjuvholm...
...,...
50582,https://onebite.app/restaurant/very-italian-pi...
50583,https://onebite.app/restaurant/donatellos-leid...
50584,https://onebite.app/restaurant/santa-lucia-lim...
50585,https://onebite.app/restaurant/pizze-pronto-sc...


In [ ]:
start = time.time()

# dave_scores = []
# user_scores = []
# name = []
# address = []

links2 = links[29101:]

for link in list(links2['link']):

    clear_output()
    print(link)
    r = requests.get(link)
    bs = BeautifulSoup(r.text)
    
    if r.status_code == 200:

        temp = []

        for elem in bs.find_all('a'):
            if 'jsx-84601126' in (elem.attrs['class']):
                temp.append(elem.text)

        name.append(temp[0])
        address.append(temp[1])

        temp2 = []

        for elem in bs.find_all('span'):
            if 'score' in (elem.attrs['class']):
                temp2.append(elem.text)

        dave_scores.append(temp2[0])
        user_scores.append(temp2[1])
        
    else:
        pass

    time.sleep(0.25)
    
end = time.time()
    
print(end-start)

In [215]:
df = pd.DataFrame({'restaurant_name': name,
                   'dave_scores': dave_scores,
                   'user_scores': user_scores,
                   'address': address})

df

,restaurant_name,dave_scores,user_scores,address
0,Pizza Hut,0.0,2.7,"213. Sok., Adana, 01"
1,Del sole,0.0,7.3,"字ニセコ483-1, 虻田郡, 01"
2,Lava Lounge Pizza Niseko,0.0,4.2,"倶知安町字山田191-19, 虻田郡, 01"
3,Pizzaria Cosa Nostra,0.0,8.5,"R. Custódio Brás Pacheco, 1, Vila Nova de Milf..."
4,Olivia Tjuvholmen,0.0,8.4,"Bryggegangen 4, Oslo, 03"
...,...,...,...,...
6359,Brickhouse Pizzeria,0.0,7.8,"820 Main St, Silt, CO"
6360,Hogback Pizza,0.0,7.3,"457 W Main St, New Castle, CO"
6361,Russos,0.0,5.9,"40 Market St, Glenwood Springs, CO"
6362,Timbo's Pizza,0.0,4.5,"23258 Two Rivers Rd, Basalt, CO"


In [293]:
len(dave_scores)

29101

In [283]:
r

<Response [200]>

In [289]:
for i in range(25):
    
    clear_output()
    print(i)
    time.sleep(0.5)

24


In [287]:
from IPython.display import clear_output

for i in range(10):
    clear_output()
    print("Hello World!")

Hello World!
